In [9]:
import requests
import csv
import folium
import pandas as pd

In [10]:
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

query = """
[out:json][timeout:120];
area["name"="Berlin"]["boundary"="administrative"]->.searchArea;
(
  node["amenity"~"^(restaurant|cafe|bar)$"](area.searchArea);
  way["amenity"~"^(restaurant|cafe|bar)$"](area.searchArea);
  relation["amenity"~"^(restaurant|cafe|bar)$"](area.searchArea);
);
out center;
"""

print("Eseguo query Overpass...")
response = requests.post(OVERPASS_URL, data={"data": query})
response.raise_for_status()
data = response.json()

records = []

for el in data["elements"]:
    tags = el.get("tags", {})
    name = tags.get("name", "N/A")
    amenity = tags.get("amenity", "N/A")
    cuisine = tags.get("cuisine", "")
    website = tags.get("website", "")
    phone = tags.get("phone", "")
    opening_hours = tags.get("opening_hours", "")
    takeaway = tags.get("takeaway", "")
    wheelchair = tags.get("wheelchair", "")
    
    addr_street = tags.get("addr:street", "")
    addr_housenumber = tags.get("addr:housenumber", "")
    addr_postcode = tags.get("addr:postcode", "")
    addr_city = tags.get("addr:city", "")
    address = f"{addr_street} {addr_housenumber}, {addr_postcode} {addr_city}".strip()

    # Coord
    if el["type"] == "node":
        lat, lon = el.get("lat"), el.get("lon")
    else:
        lat, lon = el.get("center", {}).get("lat"), el.get("center", {}).get("lon")

    records.append({
        "name": name,
        "category": amenity,
        "cuisine": cuisine,
        "address": address.strip(", "),
        "lat": lat,
        "lon": lon,
        "website": website,
        "phone": phone,
        "opening_hours": opening_hours,
        "takeaway": takeaway,
        "wheelchair": wheelchair
    })

print(f"Elementi estratti: {len(records)}")

# Salva in CSV
with open("berlin_places.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "name", "category", "cuisine", "address", "lat", "lon",
        "website", "phone", "opening_hours", "takeaway", "wheelchair"
    ])
    writer.writeheader()
    writer.writerows(records)

print("Dati salvati in berlin_places.csv")

# Visualiz
print("Creo mappa con Folium...")
m = folium.Map(location=[52.52, 13.405], zoom_start=12)

for r in records:
    if r["lat"] and r["lon"]:
        popup_html = f"""
        <b>{r['name']}</b><br>
        {r['category']} - {r['cuisine']}<br>
        {r['address']}<br>
        <a href='{r['website']}' target='_blank'>{r['website']}</a><br>
        Tel: {r['phone']}<br>
        Orari: {r['opening_hours']}
        """
        folium.Marker(
            location=[r["lat"], r["lon"]],
            popup=popup_html
        ).add_to(m)

m.save("berlin_places_map.html")

Eseguo query Overpass...
Elementi estratti: 8190
Dati salvati in berlin_places.csv
Creo mappa con Folium...


In [11]:
df = pd.read_csv("berlin_places.csv")

In [12]:
df['category'].value_counts()

category
restaurant    4721
cafe          2513
bar            956
Name: count, dtype: int64

In [13]:
df

,name,category,cuisine,address,lat,lon,website,phone,opening_hours,takeaway,wheelchair
0,Aida,restaurant,italian;pizza,"Knesebeckstraße , 10623 Berlin",52.506864,13.322859,https://www.aida-restaurant.de/,+49 30 318 06 750,Mo-Sa 11:30-21:00; Su 15:00-21:00,NaN,no
1,Madame Ngo,restaurant,asian,"Kantstraße 30, 10623 Berlin",52.506212,13.318081,NaN,+49 157 53604089,"Mo-Sa 12:00-16:30, 18:00-22:00; Su 12:00-16:30...",NaN,limited
2,Nam Thuân,restaurant,vietnamese,"Pestalozzistraße 106, 10625 Berlin",52.507320,13.320780,NaN,NaN,NaN,NaN,no
3,La Rose,restaurant,italian,"Neue Kantstraße 19, 14057 Berlin",52.506318,13.284626,NaN,NaN,NaN,NaN,NaN
4,Eiscafe Eisberg,cafe,NaN,"Brunnenstraße 55, 13355 Berlin",52.538855,13.396097,http://www.eis-berg.de/,NaN,NaN,NaN,yes
...,...,...,...,...,...,...,...,...,...,...,...
8185,Frühstücksaal,cafe,breakfast;juice;pastry,NaN,52.509261,13.260286,NaN,NaN,NaN,NaN,NaN
8186,Beach 61 Bar,bar,NaN,NaN,52.496152,13.371956,https://beach61.de,NaN,NaN,NaN,NaN
8187,WonderWaffel,cafe,waffle,NaN,52.526477,13.343583,NaN,NaN,"Su 11:30-22:00; Mo-Th 10:00-22:00; Fr,Sa 10:00...",yes,yes
8188,Panino & Coffee,cafe,italian;coffee_shop,"Motzstraße 5, 10777 Berlin",52.498592,13.351791,NaN,NaN,NaN,NaN,NaN


In [14]:
def get_bezirk(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": lat,
        "lon": lon,
        "format": "json",
        "addressdetails": 1
    }
    headers = {"User-Agent": "venue-data-cleaner/1.0"}  # Add this line
    r = requests.get(url, params=params, headers=headers, timeout=5)
    r.raise_for_status()
    data = r.json()
    return data.get("address", {}).get("suburb") \
        or data.get("address", {}).get("city_district") \
        or data.get("address", {}).get("borough")

In [15]:
import requests

def get_address(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": lat,
        "lon": lon,
        "format": "json",
        "addressdetails": 1
    }
    headers = {"User-Agent": "venue-data-cleaner/1.0"}  # Required for polite usage
    try:
        r = requests.get(url, params=params, headers=headers, timeout=5)
        r.raise_for_status()
        data = r.json()
        return data.get("display_name")  # Full formatted address
    except requests.exceptions.RequestException as e:
        print(f"Error fetching address for ({lat}, {lon}): {e}")
        return None

In [16]:
# Replace missing addresses with reverse geocoded ones
df['address'] = df.apply(
    lambda row: row['address'] if pd.notna(row['address']) else get_address(row['lat'], row['lon']),
    axis=1
)

Error fetching address for (52.474521, 13.2922768): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5)
Error fetching address for (52.522731, 13.5568419): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5)


In [28]:
df['district'] = df.apply(lambda r: get_bezirk(r['lat'], r['lon']), axis=1)

In [29]:
#show the updated DataFrame
print(df.head())
df.to_csv("berlin_places_updated.csv", index=False)

              name    category        cuisine  \
0             Aida  restaurant  italian;pizza   
1       Madame Ngo  restaurant          asian   
2        Nam Thuân  restaurant     vietnamese   
3          La Rose  restaurant        italian   
4  Eiscafe Eisberg        cafe            NaN   

                              address        lat        lon  \
0      Knesebeckstraße , 10623 Berlin  52.506864  13.322859   
1         Kantstraße 30, 10623 Berlin  52.506212  13.318081   
2  Pestalozzistraße 106, 10625 Berlin  52.507320  13.320780   
3    Neue Kantstraße 19, 14057 Berlin  52.506318  13.284626   
4      Brunnenstraße 55, 13355 Berlin  52.538855  13.396097   

                           website              phone  \
0  https://www.aida-restaurant.de/  +49 30 318 06 750   
1                              NaN   +49 157 53604089   
2                              NaN                NaN   
3                              NaN                NaN   
4          http://www.eis-berg.de/       